In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn

RANDOM_SEED = 42

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
df = pd.read_csv("./kidney-stone-dataset.csv")
df = df.drop(columns=["Unnamed: 0"])
X = df[["ph", "osmo", "urea", "calc", "gravity", "cond"]]
y = df["target"]  # or correct target column nameprint(df[:5])
X = df[["ph", "osmo", "urea", "calc", "gravity", "cond"]]
X = torch.from_numpy(np.array(X)).type(dtype=torch.float32)
y = torch.from_numpy(np.array(y)).type(dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED
)


class Kidney_Stone(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=6, out_features=24)
        self.layer2 = nn.Linear(in_features=24, out_features=12)
        self.layer3 = nn.Linear(in_features=12, out_features=1)

    def forward(self, X):
        X = self.layer1(X)
        X = self.layer2(X)
        X = self.layer3(X)
        return X


torch.manual_seed(RANDOM_SEED)
model_7 = Kidney_Stone()

In [36]:

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params=model_7.parameters(), lr=0.01)

In [ ]:
print(X_train.size())
print(len(model_7.state_dict()))
epochs = 200
for epoch in range(epochs):
    model_7.train()

    y_logits = model_7(X_train).squeeze()
    loss = loss_fn(y_logits, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model_7.eval()
    with torch.inference_mode():
        test_logits = model_7(X_test).squeeze()
        test_loss = loss_fn(test_logits, y_test)

        train_preds = torch.round(torch.sigmoid(y_logits))
        test_preds = torch.round(torch.sigmoid(test_logits))
    if epoch % 10 == 0:
        print(
            f"Epoch {epoch} | Train Loss: {loss.item():.4f} | Test Loss: {test_loss.item():.4f}"
        )

    

torch.Size([72, 6])
6
Epoch 0 | Train Loss: 0.2036 | Test Loss: 0.2534
Epoch 10 | Train Loss: 0.2030 | Test Loss: 0.2531
Epoch 20 | Train Loss: 0.2023 | Test Loss: 0.2529
Epoch 30 | Train Loss: 0.2017 | Test Loss: 0.2527
Epoch 40 | Train Loss: 0.2010 | Test Loss: 0.2526
Epoch 50 | Train Loss: 0.2004 | Test Loss: 0.2525
Epoch 60 | Train Loss: 0.1997 | Test Loss: 0.2524
Epoch 70 | Train Loss: 0.1991 | Test Loss: 0.2522
Epoch 80 | Train Loss: 0.1984 | Test Loss: 0.2521
Epoch 90 | Train Loss: 0.1978 | Test Loss: 0.2520
Epoch 100 | Train Loss: 0.1972 | Test Loss: 0.2520
Epoch 110 | Train Loss: 0.1965 | Test Loss: 0.2519
Epoch 120 | Train Loss: 0.1959 | Test Loss: 0.2518
Epoch 130 | Train Loss: 0.1953 | Test Loss: 0.2517
Epoch 140 | Train Loss: 0.1947 | Test Loss: 0.2516
Epoch 150 | Train Loss: 0.1941 | Test Loss: 0.2516
Epoch 160 | Train Loss: 0.1935 | Test Loss: 0.2515
Epoch 170 | Train Loss: 0.1929 | Test Loss: 0.2514
Epoch 180 | Train Loss: 0.1924 | Test Loss: 0.2514
Epoch 190 | Train Lo